This notebook aggregates polls for each election we're interested in for later prediction in the model of elections in Paris at the district level (see notebook `munic_model.ipynb`). For each election, polls are aggregated according to their sample size, their recency and the historical performance of the pollster at the time of the election.

In [3]:
%load_ext lab_black
%load_ext watermark

import numpy as np
import os
import pandas as pd
import scipy as sp

from typing import List

In [20]:
DB_FOLDER = "/Users/alex_andorra/Dropbox/538/DATA/dbconsolidee"
RATINGS_FOLDER = "/Users/alex_andorra/repos/contesdefaits/pollsters_ratings"

PARTIES = ["farleft", "left", "green", "center", "right", "farright", "other"]
NB_PARTIES = {
    "nbfarleft": "farleft",
    "nbleft": "left",
    "nbgreen": "green",
    "nbcenter": "center",
    "nbright": "right",
    "nbfarright": "farright",
}

VARIABLES_TO_KEEP = [
    "type",
    "dateelection",
    "date",
    "sondage",
    "samplesize",
    "nbfarleft",
    "nbleft",
    "nbgreen",
    "nbcenter",
    "nbright",
    "nbfarright",
]
BASE_VARIABLES = ["type", "dateelection", "date", "sondage", "samplesize"]

DATES_ELECTIONS = {
    "euro2009": "2009-06-07",
    "euro2014": "2014-05-25",
    "legis2007": "2007-06-10",
    "legis2012": "2012-06-10",
    "legis2017": "2017-06-11",
    "munic2014": "2014-03-23",
    "presid2007": "2007-04-22",
    "presid2012": "2012-04-22",
    "presid2017": "2017-04-23",
    "regio2010": "2010-03-14",
    "regio2015": "2015-12-06",
}

SPAN = 5  # span of poll-aggregation

Let's load the data between 2006 inclusive and 2019 exclusive (2019 will be our out-of-sample test election, and we already have the poll aggregation for this election):

In [5]:
all_polls = pd.read_csv(
    os.path.join(DB_FOLDER, "tour1_complet_unitedfl.csv"),
    parse_dates=["date", "dateelection"],
    usecols=VARIABLES_TO_KEEP,
).sort_values(["date", "sondage"])

all_polls = all_polls[
    (all_polls.sondage != "seats")
    & (all_polls.sondage != "result")
    & (all_polls.dateelection.dt.year >= 2006)
    & (all_polls.dateelection.dt.year < 2019)
].reset_index(drop=True)
all_polls

,type,dateelection,date,sondage,samplesize,nbfarleft,nbleft,nbgreen,nbcenter,nbright,nbfarright
579,legislatives,2017-06-11,2017-06-07,Elabe,1152.0,11.0,9.0,3.0,29.0,23.0,17.0
580,legislatives,2017-06-11,2017-06-07,Harris,500.0,12.0,7.0,3.0,30.0,19.0,17.0
581,legislatives,2017-06-11,2017-06-07,Ifop,886.0,11.0,8.0,3.5,30.0,20.0,18.0
582,legislatives,2017-06-11,2017-06-07,OpinionWay,1667.0,12.0,7.0,3.0,30.0,21.0,18.0
583,legislatives,2017-06-11,2017-06-08,Ipsos,1112.0,11.5,8.0,3.0,31.5,22.0,17.0


We have to add the polls for Paris city-council election in 2014 -- these are not included in our database and our pollster ratings because 1/ there aren't a lot of them and 2/ only two pollsters surveyed this race. So usually they don't hold a lot of information. But here they do: as our goal in the model will be to predict Paris 2020 city-council elections, the 2014 election is the most relevant election to date, and its associated polls -- although limited -- are of interest.

So let's load these bad boys and concatenate them to the previous polls:

In [31]:
munic2014 = pd.read_excel("data/polls_1st_round/paris_city_council_2014.xlsx")
munic2014["type"] = "municipale"
munic2014["dateelection"] = pd.to_datetime(DATES_ELECTIONS["munic2014"])

all_polls = pd.concat(
    [all_polls, munic2014], ignore_index=True, sort=False
).sort_values(["date", "sondage"])
all_polls

,type,dateelection,date,sondage,samplesize,nbfarleft,nbleft,nbgreen,nbcenter,nbright,nbfarright
0,president,2007-04-22,2006-05-18,Kantar,715.0,5.0,30.0,2.5,8.0,34.0,10.0
1,president,2007-04-22,2006-06-15,Kantar,788.0,6.0,32.0,2.0,8.0,31.0,12.5
2,president,2007-04-22,2006-07-17,Kantar,601.0,4.0,32.0,1.5,6.0,35.0,11.5
3,president,2007-04-22,2006-09-05,Kantar,683.0,3.5,34.0,1.5,7.0,36.0,10.0
4,president,2007-04-22,2006-10-05,Kantar,839.0,5.0,29.5,2.0,7.0,38.0,9.5
...,...,...,...,...,...,...,...,...,...,...,...
579,legislatives,2017-06-11,2017-06-07,Elabe,1152.0,11.0,9.0,3.0,29.0,23.0,17.0
580,legislatives,2017-06-11,2017-06-07,Harris,500.0,12.0,7.0,3.0,30.0,19.0,17.0
581,legislatives,2017-06-11,2017-06-07,Ifop,886.0,11.0,8.0,3.5,30.0,20.0,18.0
582,legislatives,2017-06-11,2017-06-07,OpinionWay,1667.0,12.0,7.0,3.0,30.0,21.0,18.0


,type,dateelection,date,sondage,samplesize,nbfarleft,nbleft,nbgreen,nbcenter,nbright,nbfarright
0,president,2007-04-22,2006-05-18,Kantar,715.0,5.0,30.0,2.5,8.0,34.0,10.0
1,president,2007-04-22,2006-06-15,Kantar,788.0,6.0,32.0,2.0,8.0,31.0,12.5
2,president,2007-04-22,2006-07-17,Kantar,601.0,4.0,32.0,1.5,6.0,35.0,11.5
3,president,2007-04-22,2006-09-05,Kantar,683.0,3.5,34.0,1.5,7.0,36.0,10.0
4,president,2007-04-22,2006-10-05,Kantar,839.0,5.0,29.5,2.0,7.0,38.0,9.5
...,...,...,...,...,...,...,...,...,...,...,...
584,municipale,2014-03-23,2013-12-15,Ifop,920.0,5.0,39.0,6.5,NaN,38.0,9.0
585,municipale,2014-03-23,2014-01-03,Ifop,931.0,5.0,39.5,6.0,NaN,38.5,9.0
586,municipale,2014-03-23,2014-01-05,CSA,810.0,5.0,38.0,7.0,NaN,39.0,8.0
587,municipale,2014-03-23,2014-01-25,Ifop,1089.0,6.0,39.5,5.5,NaN,36.5,8.0


Now let's write helper functions to compute the weights for our aggregation and then aggregate the polls election by election:

In [4]:
def compute_analyt_weights(election_date: str, df: pd.DataFrame) -> pd.DataFrame:

    pollster_ratings = pd.read_csv(
        os.path.join(RATINGS_FOLDER, f"classement_{election_date}.csv"),
    )
    print(f"Just loaded classement_{election_date}.csv\n")
    df = pd.merge(df, pollster_ratings, how="left", on="sondage")

    for p in NB_PARTIES.values():
        df[f"weightsondeur_{p}"].fillna(
            pollster_ratings[f"weightsondeur_{p}"].median(), inplace=True
        )
        df[f"analyt_weights_{p}"] = np.log(df.samplesize) * df[f"weightsondeur_{p}"]

    return df.set_index("date").sort_index()


def agg_polls(df: pd.DataFrame) -> pd.DataFrame:

    alpha = 2 / (SPAN + 1)
    unique_dates = sorted(set(df.index))

    for nb_p, p in NB_PARTIES.items():
        for i, d_outer in enumerate(unique_dates):
            for j, d_inner in enumerate(unique_dates[: i + 1]):
                df.loc[d_inner, f"expon_weights_{p}"] = (1 - alpha) ** (i - j)

            df[f"final_weights_{p}"] = (
                df[f"analyt_weights_{p}"] * df[f"expon_weights_{p}"]
            )
            final_weights = df.loc[:d_outer, f"final_weights_{p}"]
            vote_share = df.loc[:d_outer, f"{nb_p}"]

            df.loc[d_outer, f"{p}_agg"] = np.average(vote_share, weights=final_weights)

            # compute aggregate sample size only once:
            if p == "right":
                # same weights, whatever the party:
                expon_weights = df.loc[:d_outer, "expon_weights_right"]
                sample_size = df.loc[:d_outer, "samplesize"]
                df.loc[d_outer, "samplesize_agg"] = round(
                    np.average(sample_size, weights=expon_weights)
                )

    return df.reset_index()[
        ["type", "dateelection", "samplesize_agg"]
        + [f"{p}_agg" for p in NB_PARTIES.values()]
    ]

And now we just have to run these functions for all the elections we're interested in:

In [5]:
polls_series = []

for election in DATES_ELECTIONS.keys():
    election_df = compute_analyt_weights(
        election, all_polls[all_polls.dateelection == DATES_ELECTIONS[election]]
    )
    polls_series.append(agg_polls(election_df).iloc[-1])

polls_df = (
    pd.concat(polls_series, axis=1).T.reset_index(drop=True).sort_values("dateelection")
)
polls_df.to_excel("data/aggregated_polls.xlsx")
polls_df

Just loaded classement_euro2009.csv

Just loaded classement_euro2014.csv

Just loaded classement_legis2012.csv

Just loaded classement_legis2017.csv

Just loaded classement_regio2010.csv

Just loaded classement_regio2015.csv



,type,dateelection,samplesize_agg,farleft_agg,left_agg,green_agg,center_agg,right_agg,farright_agg
0,europeennes,2009-06-07,2287,12.9945,19.6148,13.3515,11.1685,27.2626,5.75632
4,regionales,2010-03-14,907,9.82754,28.9297,13.5842,4.684,28.9307,9.08352
2,legislatives,2012-06-10,1193,7.97386,32.1113,NaN,2.77511,33.6824,15.2586
1,europeennes,2014-05-25,3248,7.5563,16.6892,8.88391,9.68667,21.4976,23.2143
5,regionales,2015-12-06,1749,5.26773,22.9061,5.86065,NaN,28.1125,28.8815
3,legislatives,2017-06-11,1299,11.707,7.98006,NaN,30.3754,21.0057,17.4161


In [6]:
%watermark -a AlexAndorra -n -u -v -iv

pandas 0.25.3
scipy  1.3.1
numpy  1.17.3
AlexAndorra 
last updated: Mon Jan 20 2020 

CPython 3.7.5
IPython 7.9.0
